In [1]:
!pip3 install --upgrade git+https://github.com/huggingface/transformers.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers.git to /tmp/job.1624582.hpc/pip-req-build-jxd5qqi0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/job.1624582.hpc/pip-req-build-jxd5qqi0
  Resolved https://github.com/huggingface/transformers.git to commit 594c1610fa6243b2ffb670c49faf389ca5121939
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
You should consider upgrading via the '/usr/local/easybuild_allnodes/software/Python/3.10.4-GCCcore-11.3.0/bin/python3.10 -m pip install --upgrade pip' command.


In [ ]:
#!pip install transformers -U 

In [2]:
import os
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM
from Bio import SeqIO
import math
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# Import the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("InstaDeepAI/nucleotide-transformer-500m-human-ref")
model = AutoModelForMaskedLM.from_pretrained("InstaDeepAI/nucleotide-transformer-500m-human-ref")

/home/rxa615/.local/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
import os
os.getcwd()

'/home/rxa615'

In [5]:
os.chdir('/mnt/vstor/SOM_EPBI_XXZ10/raghav/Unsupervised_Evaluation_LLM/PaperWork/datasets')

In [10]:
datasets_folder = '/mnt/vstor/SOM_EPBI_XXZ10/raghav/Unsupervised_Evaluation_LLM/PaperWork/datasets'
csv_files = [file for file in os.listdir(datasets_folder) if file.endswith('.csv')]

In [11]:
csv_files

['test_demo_coding_vs_intergenomic_seqs.csv',
 'test_human_enhancers_cohn.csv',
 'test_human_enhancers_ensembl.csv',
 'test_human_ensembl_regulatory.csv',
 'test_human_nontata_promoters.csv',
 'test_human_ocr_ensembl.csv',
 'train_demo_coding_vs_intergenomic_seqs.csv',
 'train_human_enhancers_cohn.csv',
 'train_human_enhancers_ensembl.csv',
 'train_human_ensembl_regulatory.csv',
 'train_human_nontata_promoters.csv',
 'train_human_ocr_ensembl.csv']

In [12]:
csv_files = csv_files[6:7]
csv_files

['train_demo_coding_vs_intergenomic_seqs.csv']

In [13]:
from tqdm import tqdm


# Define a function to calculate mean embedding for a list of sequences
def calculate_mean_embeddings(sequences, tokenizer, model):
    all_mean_sequence_embeddings = []
    
    # Set max length based on the tokenizer's model maximum length
    max_length = tokenizer.model_max_length
    
    # Get the number of sequences
    num_sequences = len(sequences)
    
    # Iterate through sequences in batches
    for i in tqdm(range(0, num_sequences, batch_size)):
        # Get sequences for the current batch
        batch_sequences = sequences[i:i+batch_size]
        # Tokenize sequences
        tokens_ids = tokenizer.batch_encode_plus(batch_sequences, return_tensors="pt", padding="max_length", max_length=max_length)["input_ids"]
       
        # Compute the embeddings
        attention_mask = tokens_ids != tokenizer.pad_token_id
        torch_outs = model(
            tokens_ids,
            attention_mask=attention_mask,
            encoder_attention_mask=attention_mask,
            output_hidden_states=True
        )
        
        # Compute sequences embeddings
        embeddings = torch_outs['hidden_states'][-1].detach().numpy()
        
        # Add embed dimension axis
        attention_mask = np.expand_dims(attention_mask.numpy(), axis=-1)

        # Compute mean embeddings per sequence
        mean_sequence_embeddings = np.sum(attention_mask * embeddings, axis=-2) / np.sum(attention_mask, axis=1)
        
        # Append to the list
        all_mean_sequence_embeddings.append(mean_sequence_embeddings)
    
    # Concatenate all the mean embeddings
    all_mean_sequence_embeddings = np.concatenate(all_mean_sequence_embeddings, axis=0)
    
    return all_mean_sequence_embeddings


In [ ]:
output_folder_path = "/mnt/vstor/SOM_EPBI_XXZ10/raghav/Unsupervised_Evaluation_LLM/PaperWork/nucleotide-transformer-500m-human-ref/"


# Initialize the list to store all mean sequence embeddings
all_mean_sequence_embeddings = []

# Apply the calculate_mean_embeddings function to each CSV file
for csv_file in csv_files:
    file_path = os.path.join(datasets_folder, csv_file)
    print(csv_file)
    df = pd.read_csv(file_path)
    print(df.shape)

    tqdm.pandas()
    sequences = df['seq'].tolist()
    batch_size = 10
    mean_embeddings = calculate_mean_embeddings(sequences, tokenizer, model)

    # Save the 'mean_embedding' column as a numpy array
    embedding_file_path = os.path.join(output_folder_path, os.path.splitext(csv_file)[0] + "_NT_Embeddings.npy")
    np.save(embedding_file_path, mean_embeddings)
    print(f"Embeddings saved successfully at: {embedding_file_path}")


train_demo_coding_vs_intergenomic_seqs.csv
(75000, 2)


  0%|          | 23/7500 [07:54<42:27:35, 20.44s/it]